<a href="https://colab.research.google.com/github/Jeev-6/Agents/blob/main/Generic_Video_Narration_Tool_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Video Narration Tool Tutorial

In this tutorial, we will build a **Generic Video Narration Tool** application. This application leverages modern AI tools and APIs such as GPT-4o Realtime, OpenAI, Gradio, LangChain, and ChromaDB. The translator provides real-time sign language interpretation, educational learning modules, and community features.

We'll develop this tutorial in **Google Colab**, utilizing a step-by-step approach to explain every part of the process. The application’s workflow will involve video frame processing, GPT-4o API integration for real-time text and audio generation, and a web-based UI using Gradio.

```
# This is formatted as code
```



---

## Prerequisites
Ensure the following libraries are installed in your Google Colab environment:

In [ ]:
!pip install opencv-python gradio langchain chromadb openai requests langchain_openai

## Step 1: Import Required Libraries

In [ ]:
import cv2
import base64
import os
import openai
import requests
from IPython.display import Audio
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from google.colab import userdata

## Step 2: Set Up OpenAI API Key
To use OpenAI’s GPT-4o Realtime API, set up your API key:

In [ ]:
# Set up OpenAI API key
OPENAI_API_KEY = userdata.get('openai')  # Replace with your OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

## Step 3: Generate Descriptions Voiceover Narration Using GPT-4o

We’ll use GPT-4o to create compelling text descriptions based on the video frames. Also Create a narration script and convert it to audio using GPT-4o.

In [2]:
def translate_and_narrate(video_file):
    # Process video frames
    video = cv2.VideoCapture(video_file)  # Use video_file directly
    base64_frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64_frames.append(base64.b64encode(buffer).decode("utf-8"))
    video.release()
    client = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0.7,
        max_tokens=1000
    )

    messages = [
        SystemMessage(content="You are a helpful assistant that generates compelling video descriptions."),
        HumanMessage(content="Generate a description for this video:"),
        *[HumanMessage(content=[{
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{frame}"}
        }]) for frame in base64_frames[0::50]]
    ]

    result = client.invoke(messages)
    description = result.content  # Access the generated text using 'content'

    # Generate narration audio
    narration_prompt = f"Create a narration for the following description in a dramatic style: {description}"
    narration_result = client([HumanMessage(content=narration_prompt)])
    narration_text = narration_result.content

    audio_response = requests.post(
        "https://api.openai.com/v1/audio/speech",
        headers={"Authorization": f"Bearer {OPENAI_API_KEY}"},
        json={"model": "tts-1-1106", "input": narration_text, "voice": "onyx"}
    )
    audio_content = audio_response.content

    # # Save audio for download
    # audio_path = "narration.mp3"
    # with open(audio_path, "wb") as f:
    #     f.write(audio_content)

    # return description, audio_path

    audio_path = "narration.mp3"
    with open(audio_path, "wb") as f:
        f.write(audio_content)

    # Read audio data for returning
    with open(audio_path, "rb") as f:
        audio_data = f.read()

    return description, audio_data, audio_path  # Return audio data

## Step 4: Build the Gradio Interface
Finally, integrate all components into a Gradio interface for real-time sign language translation.

In [ ]:
# Gradio interface
def download_audio(audio_path):
    return audio_path
iface = gr.Interface(
       fn=translate_and_narrate,
       inputs=gr.Video(label="Upload a Video"),
       outputs=[
           gr.Text(label="Video Description"),
           gr.Audio(label="Narrated Audio", interactive=False),
           gr.File(label="Download Narrated Audio", file_types=[".mp3"])
       ],
       live=True,
       title="Generic Video Narration Tool"
   )

iface.queue().launch(share=True, debug=True)  # Add queue() before launch

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://189e7f93fdc99cf075.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://189e7f93fdc99cf075.gradio.live


## Step 8: Monetization Opportunities
1. **Educational Institutions**: Provide accessibility features to schools for teaching sign language.
2. **Accessibility Services**: Integrate into platforms that aid people with hearing impairments.
3. **Learning Modules**: Offer sign language learning programs to individuals.

---

## Conclusion
This tutorial demonstrates how to build a real-time sign language translator using cutting-edge AI technologies. By leveraging GPT-4o Realtime APIs, Gradio, and OpenCV, we created a fully functional application for accessibility and learning. Future enhancements could include multi-language support, real-time feedback loops, and user authentication.